Imports

In [ ]:
from openai import OpenAI
import requests
from urllib.parse import quote
import json
import sys
import urllib
import argparse
from urllib.parse import quote
from requests.exceptions import JSONDecodeError
from __future__ import print_function
import pprint
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Main function

In [ ]:
def main():
    """
    The main function that interacts with the user to answer restaurant-related questions.

    This function continuously prompts the user for restaurant-related questions until the user enters "exit".
    It then processes the user's question by reducing it to a search term, searching for restaurants on TripAdvisor
    and Yelp using the search term, combining the search results, and finally obtaining an answer to the user's question
    using the OpenAI API.

    """
    while True:
        question = input("Ask me a restaurant-related question: ")
        if question.lower() == "exit":
            print("Exiting...")
            break

        search_term = reduce_to_search_term(question)

        # Search on TripAdvisor
        location_ids, location_names = tripAdvisorAPISearch(tripAPI_JOSH, search_term)
        trip_context = {}
        if location_ids:
            print("Found restaurants on TripAdvisor")
            for location_id in location_ids:
                reviews = tripAdvisorAPIReviews(tripAPI_JOSH, location_id)
                trip_context[location_id] = reviews
            tripcompletepackage = list(zip(location_ids, location_names,trip_context))

        else:
            print("No restaurants found on TripAdvisor.")

        # Search on Yelp
        yelp_response = yelp_search(API_KEY, search_term, "Cincinnati")
        yelp_context = {}
        if yelp_response:
            print("Found restaurants on Yelp")
            yelp_context = yelp_response
            #print.pprint(yelp_response)
        else:
            print("No restaurants found on Yelp.")

        combined_context = {
            #"NameFromTrip":location_names,
            "TripAdvisor": tripcompletepackage,
            "Yelp": yelp_context
        }

        answer = ask_question_openai_api(question, combined_context)
        print("Answer:", answer)

if __name__ == "__main__":
    main()

Prompt OpenAI API function

In [ ]:
API_KEY = ""

def ask_question_openai_api(question, context):
    """
    Utilizes the OpenAI API to generate a response to a given question based on the provided context.

    Args:
        question (str): The question to be answered.
        context (dict): The context in which the question is being asked, including information about restaurants.

    Returns:
        str: The generated response to the question, provided by the OpenAI API.

    """
    # Set up OpenAI API client with your API key
    client = OpenAI(api_key=API_KEY)

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "user",
                "content": f"""Based only off this context: {context}. 
                To the best of your ability, provide a recommendation/answer to this question: {question}.
                Again, make sure to only answer using the given context,
                and only suggest restaurants located in the Cincinnati area
                (all restaurants given in context are from the Cincinnati area).
                Please answer the question by stating 'To my knowledge, ~Your answer~ and give a detailed explanation.'"""
            }
        ],
    )
    return response.choices[0].message.content

Algorithm to reduce question to a search term using NLP

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

def reduce_to_search_term(question):
    """
    Reduce a question to a single search term relevant for food or cuisine.

    Args:
        question (str): The question to be reduced.

    Returns:
        str: The reduced search term.

    """
    # Tokenize the question
    tokens = word_tokenize(question)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

    # Define a list of common food and cuisine words
    food_cuisine_words = ['italian', 'indian', 'eatery', 'chinese', 'korean', 'barbecue', 'american', 'diner', 'cafe',
                      'pizza', 'burger', 'sushi', 'taco', 'pasta', 'steak',
                      'chicken', 'salad', 'sandwich', 'soup', 'noodle', 'curry',
                      'french', 'mediterranean', 'thai', 'mexican', 'japanese', 'seafood', 'vegetarian', 'vegan',
                      'gluten-free', 'spanish', 'greek', 'vietnamese', 'fusion', 'fast food', 'pub', 'bbq',
                      'buffet', 'fine dining', 'street food', 'ramen', 'tapas', 'bagel', 'bistro',
                      'creole', 'caribbean', 'hawaiian', 'german', 'scandinavian', 'soul food', 'dim sum',
                      'moroccan', 'afghan', 'peruvian', 'cuban', 'malaysian', 'polish', 'ethiopian', 'turkish',
                      'irish', 'russian', 'brazilian', 'argentinian', 'filipino', 'african', 'middle eastern',
                      'indonesian', 'bangladeshi', 'nepalese', 'korean barbecue', 'szechuan', 'hot pot', 'teppanyaki']

    # Calculate term frequencies
    term_freq = {}
    for word in filtered_tokens:
        # Assign higher weight to food and cuisine words
        weight = 1
        if word in food_cuisine_words:
            weight = 2  # Increase weight for food and cuisine words

        term_freq[word] = term_freq.get(word, 0) + weight

    # Select the most frequent term
    max_freq = 0
    search_term = ''
    for term, freq in term_freq.items():
        if freq > max_freq:
            max_freq = freq
            search_term = term

    return search_term

Trip Adivisor API call functions

In [ ]:
def tripAdvisorAPISearch(api_key, search_term):
    """
    Search for locations on TripAdvisor based on a search term.

    Args:
        api_key (str): The API key for authorization.
        search_term (str): The term to search for locations.

    Returns:
        list: A list of TripAdvisor location IDs relevant to the search term.

    """
    url = 'https://api.content.tripadvisor.com/api/v1/location/search?key=' + api_key + '&searchQuery=' + search_term + '&category="restaurants"&latLong="39.1031%2C%84.512"&radius=15&radiusUnit="mi"&language=en' # construct url for API call
    headers = {"accept": "application/json"} # set headers
    response = requests.get(url, headers=headers) # API call

    # Format and filter response data
    response_data = response.json()
    response_data = response_data['data']
    locationIDs = []
    names = []
    for element in response_data: # Pull each location ID out of results
        locationIDs.append(element['location_id'])
        names.append(element['name'])
    return locationIDs, names

def tripAdvisorAPIReviews(api_key, locationID):
    """
    Get reviews from TripAdvisor based on a location ID.

    Args:
        api_key (str): The API key for authorization.
        locationID (str): The ID of the location to get reviews for.

    Returns:
        list: A list of raw text reviews for the inputted location ID.

    """
    url = "https://api.content.tripadvisor.com/api/v1/location/" + str(locationID) + "/reviews?key=" + api_key + '&language=en' # construct url for API call
    headers = {"accept": "application/json"} # set headers
    response = requests.get(url, headers=headers) # API call

    # Format and filter response data
    response_data = response.json()
    response_data = response_data['data']
    reviewText = []
    for element in response_data: # Pull raw text of review(s) from results
        reviewText.append(element['text'])
    return reviewText

Yelp API call functions

In [ ]:
def yelp_request(host, path, api_key, url_params=None):
    """
    Send a GET request to the Yelp API.

    Args:
        host (str): The API host URL.
        path (str): The API endpoint path.
        api_key (str): Yelp API key for authorization.
        url_params (dict): Parameters to be included in the request URL.

    Returns:
        dict: JSON response from the API.

    """
    url_params = url_params or {}
    url = f"{host}{path}"
    headers = {
        'Authorization': f'Bearer {api_key}',
        'accept': 'application/json'
    }
    try:
        response = requests.get(url, headers=headers, params=url_params)
        response.raise_for_status()  # Check for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

def yelp_search(api_key, term, location):
    """
    Query the Yelp API by a search term and location.

    Args:
        api_key (str): Yelp API key for authorization.
        term (str): The search term.
        location (str): The location to search for businesses.

    Returns:
        dict: JSON response containing search results.

    """

    SEARCH_PATH = '/v3/businesses/search'
    API_HOST = 'https://api.yelp.com'

    url_params = {
        'term': term,
        'location': location,
        'limit': 5
    }
    return yelp_request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)